In [8]:
import os
import urllib.request
import xmltodict
import pandas as pd
import plotly.plotly as py
import plotly
!pip install geopy
from geopy.geocoders import Nominatim

APIKEY='wnogOGBcdeuw2gYJb7bx'
plotly.tools.set_credentials_file(username='AnyUserNameICanUse', api_key=APIKEY)
flight_list=[]
flight_df=pd.DataFrame()
loc_data={}
flight_data={}

def download_data():
    try:
        response = urllib.request.urlopen('http://syrairport.org/flightdata/data.xml')

        XML_FILE = open('data.xml', 'w')

        XML_FILE.write(response.read().decode('utf-8'))
        XML_FILE.close()
    
    # internet is broken
    except requests.exceptions.RequestException as e: 
        print("ERROR: Cannot connect to ", url)
        print("DETAILS:", e)
        
    # response not ok
    except requests.exceptions.HTTPError as e:
        print("ERROR: Response from ", url, 'was not ok.')
        print("DETAILS:", e)


def parse_data():
    XML_FILE = open('data.xml')
    doc = xmltodict.parse(XML_FILE.read())

    list = []
    for entry in doc['flights']['flight']:
        list.append(dict(entry))

    return list

def parse_airports():
    long_lat('Syracuse')
    for i in range(len(flight_list)):
        long_lat(flight_list[i]['@city'])

def long_lat(loc):
    if loc not in loc_data:
        geolocator = Nominatim()
        location = geolocator.geocode(loc)
        loc_data[loc] = [location.latitude, location.longitude]


def plot():
    df_airports = pd.read_csv('2011_february_us_airport_traffic.csv')
    df_airports = df_airports.head()

    df_flight_paths = pd.read_csv('2011_february_aa_flight_paths.csv')
    df_flight_paths = df_flight_paths.head()

    airports = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_airports['long'],
            lat = df_airports['lat'],
            hoverinfo = 'text',
            text = df_airports['airport'],
            mode = 'markers',
            marker = dict(
                size=2,
                color='rgb(255, 0, 0)',
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                )
            ))]
    print(airports)
    
    flight_paths = []
    for i in range( len( df_flight_paths ) ):
        flight_paths.append(
            dict(
                type = 'scattergeo',
                locationmode = 'USA-states',
                lon = [ df_flight_paths['start_lon'][i], df_flight_paths['end_lon'][i] ],
                lat = [ df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i] ],
                mode = 'lines',
                line = dict(
                    width = 1,
                    color = 'red',
                ),
                opacity = float(df_flight_paths['cnt'][i])/float(df_flight_paths['cnt'].max()),
            )
        )
    print(flight_paths)
        
    layout = dict(
            title = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
            showlegend = False, 
            geo = dict(
                scope='north america',
                projection=dict( type='azimuthal equal area' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
        
    fig = dict( data=airports + flight_paths, layout=layout )
    py.iplot( fig, filename='gfukcing' )


def draw():
    airports=[]
    for i in range( len( loc_data ) ):
            airports.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = pd.Series(list(loc_data.values())[i][1]),
                    lat = pd.Series(list(loc_data.values())[i][0]),
                    hoverinfo = 'text',
                    text = pd.Series(list(loc_data.keys())[i]),
                    textposition='bottom',
                    mode = 'markers+text',
                    marker = dict(
                        size=6,
                        color='rgb(255, 0, 0)',
                        line = dict(
                            width=3,
                            color='rgba(68, 68, 68, 0)'
                        )
                )))

    flight_paths = []
    for i in range( len( flight_list ) ):
        flight_paths.append(
            dict(
                type = 'scattergeo',
                locationmode = 'USA-states',
                lon = [ loc_data[flight_list[i]['@city']][1], loc_data['Syracuse'][1] ],
                lat = [ loc_data[flight_list[i]['@city']][0], loc_data['Syracuse'][0] ],
                hoverinfo = 'text',
                text = pd.Series(str(flight_list[i]['@airlinecode'] + flight_list[i]['@flightnumber'])),
                textposition='center',
                mode = 'lines',
                line = dict(
                    width = 2,
                    color = 'red',
                )
            )
        )


    layout = dict(
                title = 'All Flight Departs from SYR',
                showlegend = False, 
                geo = dict(
                    scope='north america',
                    projection=dict( type='azimuthal equal area' ),
                    showland = True,
                    landcolor = 'rgb(243, 243, 243)',
                    countrycolor = 'rgb(204, 204, 204)',
                ),
            )
    fig = dict( data=airports + flight_paths, layout=layout)
    py.iplot( fig, filename='glhf' )

download_data()
flight_list = parse_data()
parse_airports()
draw()


You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
